# 1.Input Libraries

In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# 2. Load Data

In [35]:
column_names = [
    "erythema", "scaling", "definite borders", "itching", "koebner phenomenon",
    "polygonal papules", "follicular papules", "oral mucosal involvement", "knee and elbow involvement",
    "scalp involvement", "family history", "melanin incontinence", "eosinophils in the infiltrate",
    "PNL infiltrate", "fibrosis of the papillary dermis", "exocytosis", "acanthosis",
    "hyperkeratosis", "parakeratosis", "clubbing of the rete ridges", "elongation of the rete ridges",
    "thinning of the suprapapillary epidermis", "spongiform pustule", "munro microabcess",
    "focal hypergranulosis", "disappearance of the granular layer", "vacuolisation and damage of basal layer",
    "spongiosis", "saw-tooth appearance of retes", "follicular horn plug", "perifollicular parakeratosis",
    "inflammatory monoluclear inflitrate", "band-like infiltrate", "Age", "class"
]

dermatology_data = pd.read_csv('dermatology/dermatology.data', header=None, names=column_names)

# 3. Data Preprocessing
## 3.1 Data replace and fill missing values
- Replace the missing values with NaN
- Convert the 'Age' column to numeric
- Fill the missing values in 'Age' with the median age
- Drop the 'perifollicular parakeratosis' column(because all values are 0)

In [36]:
# Replacing the missing values with NaN
dermatology_data.replace("?", pd.NA, inplace=True)
dermatology_data['Age'] = pd.to_numeric(dermatology_data['Age'], errors='coerce')

# Filling the missing values in 'Age' with the median age
age_median = dermatology_data['Age'].median()
dermatology_data['Age'].fillna(age_median, inplace=True)

# Dropping the 'perifollicular parakeratosis' column
dermatology_data.drop('perifollicular parakeratosis', axis=1, inplace=True)

## 3.2 Data Scaling

In [37]:
# Scaling the data
dermatology_data_scaled = dermatology_data.copy()
for column in dermatology_data.columns[:-1]:  
    col_min = dermatology_data[column].min()
    col_max = dermatology_data[column].max()
    dermatology_data_scaled[column] = (dermatology_data[column] - col_min) / (col_max - col_min)

## 3.3 Data Splitting
- Split the data into features and target
- One-hot encode the target, to get 6 output nodes
- Split the data into train and test sets

In [38]:

# Splitting the data into features and target
X = dermatology_data_scaled.drop('class', axis=1).values
y = dermatology_data_scaled['class'].values

# One-hot encoding the target, to get 6 output nodes, it will more fit to train ANN model
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

/Users/zhaochengxin/anaconda3/envs/Learn/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# 4. ANN Model
## 4.1 Tools preparation
- I choose use sigmoid as my activation function
- I choose use cross-entropy as my loss function
- Due to the output is multi-class, I choose use softmax as my activation function for output layer

In [39]:
# Sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid function
def sigmoid_derivative(x):
    return x * (1 - x)

# Design the Softmax activation function for the output layer
def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

# Function to compute the loss using cross-entropy
def cross_entropy_loss(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred))

## 4.2 ANN implementation

In [40]:
class ANN:
    def __init__(self, input_size, hidden_size, output_size):
        """
        - I choose use 2 layers ANN model, the first layer is hidden layer, the second layer is output layer. I used the random randn function to initialize the weights to get more accuracy and help the model to fitting the data better.
        - For the bias, I used the zeros function to initialize the bias.
        """
        self.weights1 = np.random.randn(input_size, hidden_size)
        self.bias1 = np.zeros((1, hidden_size))
        self.weights2 = np.random.randn(hidden_size, output_size)
        self.bias2 = np.zeros((1, output_size))

    def feedforward(self, X):
        # Forward pass
        self.layer1 = sigmoid(np.dot(X, self.weights1) + self.bias1)
        self.output = softmax(np.dot(self.layer1, self.weights2) + self.bias2)
        return self.output

    def backpropagation(self, X, y, learning_rate):
        # Backward pass
        output_error = self.output - y
        output_delta = output_error

        layer1_error = output_delta.dot(self.weights2.T)
        layer1_delta = layer1_error * sigmoid_derivative(self.layer1)

        # Update weights and biases
        self.weights2 -= self.layer1.T.dot(output_delta) * learning_rate
        self.bias2 -= np.sum(output_delta, axis=0, keepdims=True) * learning_rate
        self.weights1 -= X.T.dot(layer1_delta) * learning_rate
        self.bias1 -= np.sum(layer1_delta, axis=0, keepdims=True) * learning_rate

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            self.feedforward(X)
            self.backpropagation(X, y, learning_rate)
            if epoch % 100 == 0:
                loss = cross_entropy_loss(y, self.output)
                print("Epoch:", epoch, "Loss:", loss)

In [41]:
# Creating the ANN
ann = ANN(X_train.shape[1], 10, y_train.shape[1])

# Training the ANN
ann.train(X_train, y_train, epochs=1000, learning_rate=0.01)

Epoch: 0 Loss: 1169.3902019077527
Epoch: 100 Loss: 26.77995155969457
Epoch: 200 Loss: 13.176791701519704
Epoch: 300 Loss: 8.196715266703963
Epoch: 400 Loss: 5.736662461762528
Epoch: 500 Loss: 4.334646353586688
Epoch: 600 Loss: 3.4431861991611816
Epoch: 700 Loss: 2.830376220235971
Epoch: 800 Loss: 2.3857392448008072
Epoch: 900 Loss: 2.0502675909660284


In [42]:
# Function to predict classes using the trained ANN
def predict(X, model):
    output = model.feedforward(X)
    predictions = np.argmax(output, axis=1)
    return predictions

# Predicting classes on the test set
y_pred = predict(X_test, ann)

# Converting one-hot encoded test labels back to class labels for comparison
y_test_labels = np.argmax(y_test, axis=1)

# Calculating the accuracy
accuracy = np.mean(y_pred == y_test_labels)
accuracy

0.972972972972973